## Visualize misclassified images

In [ ]:
from keras.models import load_model
img_rows, img_cols = 224, 224

# set job id
JOB_ID = 'XXXXXXXXXXXXX'

model = load_model('/mnt/training_jobs/{}/model.h5'.format(JOB_ID))
model.summary()

In [ ]:
import io
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import preprocess_input
from abeja.datasets import Client

def load_dataset_from_api(dataset_id):
    client = Client()
    dataset = client.get_dataset(dataset_id)

    for item in dataset.dataset_items.list(prefetch=True):
        file_content = item.source_data[0].get_content()
        label = item.attributes['classification'][0]['label_id']
        file_like_object = io.BytesIO(file_content)
        img = load_img(file_like_object, target_size=(img_rows, img_cols))
        yield img, label

In [ ]:
# set dataset id
dataset_id = 'XXXXXXXXXXXXX'
data = list(load_dataset_from_api(dataset_id))

In [ ]:
import numpy as np
x = np.array([img_to_array(_[0]) for _ in data])
y = np.array([_[1] for _ in data])

In [ ]:
import keras
x_preprocessed = preprocess_input(x, mode='tf')
x_preprocessed = x_preprocessed.astype('float32')
num_classes = 5

In [ ]:
results = model.predict(x_preprocessed, batch_size=1, verbose=1)

In [ ]:
predicted_classes = results.argmax(axis=-1)

In [ ]:
predicted_classes

In [ ]:
!pip install seaborn
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

class_names = ['compressor', 'fan_motor', 'filter', 'louver_motor', 'thermistor']

def print_cmx(y_true, y_pred):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=class_names, columns=class_names)
 
    plt.figure(figsize = (12,7))
    sn.heatmap(df_cmx, annot=True, fmt='g' ,square = True,cmap="Blues",)
    plt.show()
 
print_cmx(y, predicted_classes)

In [ ]:
from PIL import Image

for i, v in enumerate(predicted_classes):
    answer = y[i]
    img = data[i][0]
    if answer == v:
        continue 
    print('predict:{} answer: {}'.format(class_names[v], class_names[answer]))
    display(img)